Tutorial on how to use syslib. Currently available: calibration feature only.

In [5]:
import numpy as np
from syslibrary import syslib

Let's initiate a Calibration class and check it out 

In [6]:
calib=syslib.Calibration()

In [7]:
calib

Calibration:
  cXnu1: 1.0
  cYnu2: 1.0

calib takes two input arguments. Those are supposed to be the calibration factors, one for each field X,Y and frequency nu1,nu2. E.g., calibration factor for field T measured at frequency 95GHz. 

In [8]:
calib.defaults

{'cXnu1': 1.0, 'cYnu2': 1.0}

We can change the defaults value via the set_defaults function

In [9]:
calib.set_defaults(cXnu1=3.)
calib

Calibration:
  cXnu1: 3.0
  cYnu2: 1.0

Let's now look at the output. The input arguments can be passed as two scalars. In this case, the output is a scalar.

In [10]:
calib(cXnu1=2.,cYnu2=3.)

array([6.])

Or as an array and a scalar. Note that order matters. In this case, the output is a column vector.

In [11]:
calib(cXnu1=[1.,2.])

array([[1.],
       [2.]])

In this other case, where cXnu1 is a scalar and cYnu2 is an array, the output is a row vector.

In [12]:
calib(cYnu2=[1.,2.])

array([3., 6.])

The most general case, when both cXnu1 and cYnu2 are arrays, gives a matrix as an output.

In [13]:
calib(cXnu1=[1.,2.],cYnu2=[3.,4.])

array([[3., 4.],
       [6., 8.]])

Suppose now that cT=[1,2,3] is the array of calibration factors of T maps for a set
of 3 frequencies (say, 95, 150, 220 GHz). cE=[4,5,6] is the analogue for E maps. Let's call calib() with these two input arguments.

In [14]:
cT=[1.,2.,3.]
cE=[4.,5.,6.]
calib(cXnu1=cT, cYnu2=cE)

array([[ 4.,  5.,  6.],
       [ 8., 10., 12.],
       [12., 15., 18.]])

Each entry of this matrix corresponds to the calibration factor of the TE auto- and cross-spectra.
E.g., element (0,0) is the calibration factor of the 95x95 TE spectrum.
Element (0,1) is the calibration factor of the 95x150 spectrum, i.e. T from 95 and E from 150.
Note that it is different from element (1,0), which is the cal factor for TE 150x95,
i.e. T from 150 and E from 95.

Let's now have a look at the possibility to upload templates from file. We have defined a class called ReadTemplateFromFile. Note that the input file is in yaml format. The class takes a root name in input, check for the file, open it.

In [17]:
tfromf=syslib.ReadTemplateFromFile(rootname='test_template')

In [18]:
tfromf

ReadTemplateFromFile:
  amp: 1.0
  ell: null

tfromf can be instancieted passing the ell range and the normalisation amp as input. It will return a dictionary of cls as follows: cl[spec,f1,f2], where spec=tt,te,ee, and f1,f2=each of the experiment's channels

In [19]:
ell=np.arange(20)
clt=tfromf(ell=ell)
for k in clt.keys():
    print(k,clt[k])

('ee', 93, 93) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 93, 145) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 93, 225) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 145, 93) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 145, 145) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 145, 225) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 225, 93) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 225, 145) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('ee', 225, 225) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('te', 93, 93) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('te', 93, 145) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('te', 93, 225) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
('te', 145, 93) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

Now, let's have a look at the T-to-E leakage template implemented in the library. It takes inspiration from the Planck 2015 T-to-E beam leakage model, where:

aE(l)=aE(l)+eps(l)aT

eps(l)=eps0+eps2* l** 2 +eps4* l**4

and

TE=TE+eps(l)TT

EE=EE+2eps(l)TE+eps(l)^2*TT

There is a class called Residual(), which is the master for many kinds of templates (many more to come). residual takes 2 inputs: an array of multipoles ell, and a dictionary of cls spectra. spectra must be in the following format:
spectra[spec,f1,f2], where spec=tt,te,ee, and f1,f2 are each of the instrument's channels.
The class TtoEleak_Planck15 is a specific instance, implementing the quartic template function described above. It takes two inputs: an array of frequencies nu, and a dictionary of leakage coefficients enu. 

for each nui in nu, enu is the following:
enu[nui] is an array of 3 elements, corresponding to the 0th, 2nd, and 4th coefficients of the quartic function eps(l).

The output is a dictionary of cls, same format as spectra.

In [20]:
ell=np.arange(2,10)
cl=dict()
nu=[90,150]
cl["tt",90,90]=1.*np.ones(len(ell))#/ell**2
cl["tt",90,150]=1.*np.ones(len(ell))#5#+1./ell**2
cl["tt",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2
cl["tt",150,150]=1.*np.ones(len(ell))#25#+1./ell**2
cl["te",90,90]=1.*np.ones(len(ell))#/ell**2/10.
cl["te",90,150]=1.*np.ones(len(ell))#5#+1./ell**2/10.
cl["te",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2/10.
cl["te",150,150]=1.*np.ones(len(ell))#25#+1./ell**2/10.
cl["ee",90,90]=1.*np.ones(len(ell))#/ell**2/1000.
cl["ee",90,150]=1.*np.ones(len(ell))#5#+1./ell**2/1000.
cl["ee",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2/1000.
cl["ee",150,150]=1.*np.ones(len(ell))#25#+1./ell**2/1000.
enu1={}
enu1[90]=np.array([1.,2.,3.])
enu1[150]=np.array([4.,5.,6.])

In [21]:
teleak=syl.TtoEleak_Planck15(ell=ell,spectra=cl)
teleak

TtoEleak_Planck15:
  enu:
    '100':
    - 0.0
    - 0.0
    - 0.0
  nu: null

In [22]:
x=teleak(nu=nu,enu=enu1)
for k in x.keys():
    print(k,x[k])

('tt', 90, 90) [0. 0. 0. 0. 0. 0. 0. 0.]
('te', 90, 90) [   57.   262.   801.  1926.  3961.  7302. 12417. 19846.]
('ee', 90, 90) [3.36300000e+03 6.91680000e+04 6.43203000e+05 3.71332800e+06
 1.56974430e+07 5.33338080e+07 1.54206723e+08 3.93903408e+08]
('tt', 90, 150) [0. 0. 0. 0. 0. 0. 0. 0.]
('tt', 150, 90) [0. 0. 0. 0. 0. 0. 0. 0.]
('te', 90, 150) [  120.   535.  1620.  3879.  7960. 14655. 24900. 39775.]
('ee', 90, 150) [7.01700000e+03 1.40967000e+05 1.30004100e+06 7.47675900e+06
 3.15414810e+07 1.07032767e+08 3.09220617e+08 7.89434271e+08]
('te', 150, 90) [  120.   535.  1620.  3879.  7960. 14655. 24900. 39775.]
('ee', 150, 90) [7.01700000e+03 1.40967000e+05 1.30004100e+06 7.47675900e+06
 3.15414810e+07 1.07032767e+08 3.09220617e+08 7.89434271e+08]
('tt', 150, 150) [0. 0. 0. 0. 0. 0. 0. 0.]
('te', 150, 150) [  120.   535.  1620.  3879.  7960. 14655. 24900. 39775.]
('ee', 150, 150) [1.46400000e+04 2.87295000e+05 2.62764000e+06 1.50543990e+07
 6.33775200e+07 2.14798335e+08 6.20059800e

Another available template is a refinement of the Calibration scheme described above. Calibration_Planck15 is an instance of residual. It takes ell and spectra in input, and can be initiated with an array of frequencies nu and two dictionaries of calibration parameters cal1,cal2.

In [24]:
cal15=syslib.Calibration_Planck15(ell=ell,spectra=cl) # does not exist
cal15

AttributeError: module 'syslibrary.syslib' has no attribute 'Calibration_Planck15'

In [ ]:
cal1={}
cal1['tt']=[2.,4.]
cal1['ee']=[3.,5.]
cal15(cal1=cal1,cal2=cal1,nu=nu)

Debug below this line. Not relevant.
------------------------------------------

In [ ]:
tfromf=syl.TemplatesFromFiles(nu=['93','145','225'],root='generic_template')
tfromf

tfromf can be instancieted giving it as input the ell range and the normalisation amp. It will return as output a 3X3 array of cls. Each cl will have length equal to ell.

In [ ]:
ell=np.arange(2,6000)
tfromf(ell=ell)

Note that amp can be either a scalar, a 3d array, or a 3X3 matrix.

In [ ]:
ell=np.arange(2,6000)
tfromf(ell=ell,amp=[[1,2,3],[4,5,6],[7,8,9]])

In [ ]:
tfromf=syl.TemplatesFromFiles(nu=['93','145','225'])

In [ ]:
tfromf

In [ ]:
ell=np.arange(2,10000)
cl=dict()
nu=[90,150]
cl["tt",90,90]=1.*np.ones(len(ell))#/ell**2
cl["tt",90,150]=1.*np.ones(len(ell))#5#+1./ell**2
cl["tt",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2
cl["tt",150,150]=1.*np.ones(len(ell))#25#+1./ell**2
cl["te",90,90]=1.*np.ones(len(ell))#/ell**2/10.
cl["te",90,150]=1.*np.ones(len(ell))#5#+1./ell**2/10.
cl["te",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2/10.
cl["te",150,150]=1.*np.ones(len(ell))#25#+1./ell**2/10.
cl["ee",90,90]=1.*np.ones(len(ell))#/ell**2/1000.
cl["ee",90,150]=1.*np.ones(len(ell))#5#+1./ell**2/1000.
cl["ee",150,90]=1.*np.ones(len(ell))#5+1.#/ell**2/1000.
cl["ee",150,150]=1.*np.ones(len(ell))#25#+1./ell**2/1000.
enu1={}
enu1[90]=np.array([1.,2.,3.])
enu1[150]=np.array([4.,5.,6.])

In [ ]:
teleak=syl.TtoEleak_Planck15(ell=ell,spectra=cl)

In [ ]:
teleak

In [ ]:
x=teleak(nu=nu,enu=enu1)

In [ ]:
x['te',90,90]

In [ ]:
syl.residual(ell=ell,spectra=cl)

In [ ]:
cal15=syl.Calibration_Planck15(ell=ell,spectra=cl)

In [ ]:
cal15

In [ ]:
cal1={}
cal1['tt']=[2.,4.]
cal1['ee']=[3.,5.]

In [ ]:
cal15(cal1=cal1,cal2=cal1,nu=nu)

In [ ]:
for k in cal1.keys():
    print(np.array(cal1[k]))

In [ ]:
cl2

In [ ]:
template={}
yy={}
template['bb']=yy
yy['ciccio']=3
template.keys()

In [ ]:
cl.keys()

Debug below this line. Not relevant.
------------------------------------------

In [ ]:
import numpy as np
from itertools import product

map_names = np.array(['93','145','225'])
ell=np.arange(10000)
dl=np.ones(len(ell))
data=np.column_stack((ell,dl))
corr=product(map_names,map_names)
for i,c in enumerate(corr):
    print(i,c)
    idx = (i%3, i//3)
    print(idx)
    root='/Users/martina/Documents/University/Projects/sys_sandbox/newsysV0.1_folder/syslibrary/sysspectra/data/'
    fname=root+'cl_generic_template_'+c[0]+'_'+c[1]+'.dat'
    print(fname)
    np.savetxt(fname,data)


In [ ]:
cl2=dict()
cl2['tt']=dict()
cl2['tt'][93]=dict()
cl2['tt'][145]=dict()
cl2['tt'][225]=dict()
cl2['te']=dict()
cl2['te'][93]=dict()
cl2['te'][145]=dict()
cl2['te'][225]=dict()
cl2['ee']=dict()
cl2['ee'][93]=dict()
cl2['ee'][145]=dict()
cl2['ee'][225]=dict()
#print(cl.keys())
for k in cl2.keys():
    for f1 in cl2[k].keys():
        for f2 in [93,145,225]:
            cl2[k][f1][f2]=np.ones(len(ell)).tolist()

In [ ]:
import yaml
with open('test_template.yaml', 'w') as file:
    documents = yaml.dump(cl2, file,default_flow_style=False)

In [ ]:
with open('test_template.yaml') as file:
    documents = yaml.full_load(file)

In [ ]:
documents.keys()

In [ ]:
dcl=dict()
for spec in documents.keys():
    for f1 in documents[spec].keys():
        for f2 in documents[spec][f1].keys():
            dcl[spec,f1,f2] = np.array(documents[spec][f1][f2])

In [ ]:
print(dcl)

In [ ]:
l=np.arange(1000)
ff=syl.ReadTemplateFromFile(rootname='test_template')

In [ ]:
cl3=ff(ell=l)

In [ ]:
cl3.keys()

In [ ]:
np.shape(cl3['ee', 90, 90])